# Radioactive Decay Interactives

This notebook has embedded in it code for interactive investigation of radioative decay.  

The eventual goal is the development of two ipywidgets:
- Radioactive Decay model showing decay of a population of atoms over time. **[Sam is continuing development of this one.]**
- Isochron dating model **[Juan is tackling this one.]**

*Note:* The idea for the isochron dating interactive came from a Isochron Diagram Java app at ScienceCourseware.org.  However that app had some issues in that it didn't divide by a non-radiogenic isotope (or at least didn't mention it).  In fact, they used D_i for the initial amount of daughter isotope instead of the non-radiogenic isotope of the same element as the daughter isotope.

In [2]:
# UNcomment to turn on autoreloading with reach execution
# %load_ext autoreload
# %autoreload 2

from IPython.display import display
import numpy as np
import bqplot as bq
import ipywidgets as widgets
import pandas as pd
from math import ceil, floor, log10


Assuming a non-radiogenic isotope (that is, an isotope that is not the result of radioactive decay) that also will not decay, its amount should be constant.  This means that for different mineral samples we can measure the ratio of parent isotope versus the non-radiogenic isotope (P/D_i) and daughter isotope (D) versus the non-radiogenic isotope (D/D_i) to build an isochron plot.  For example, using the following isotopes

- D_i (non-radiogenic isotope of daughter element)
- D (Daughter Isoptope)
- P (Parent isotope)

an isochron plot could plot D/D_i versus P/D_i.  

What sets the *isochron method* (also known as the *geochron method*) apart from the just measuring parent and daughter abundances is the use of the non-radiogenic isotope of the daughter element.  This avoids the assumption of no initial daughter isotope before the rock solidified (radioactive decay can occur while rock is molten).

Some minerals in the rock incoprorate parent better than daughter which is why the initial amount of parent 
isotope versus daughter isotope can vary.  We expect daughter versus non-radiogenic isotope ratio to be constant
if we pick the non-radiogenic isotope to be the same element as the daughter isotope.

With all this said, it is actually often not this simple as many daughter isotopes are themselves radioactive and decay, leading to a chain of reactions, so comparing abundances of parent to daughter isotopes is not simple.

In [3]:
##
## Define the various isotopes we want to consider
##

isotope_info = pd.DataFrame(columns=['Name', 'PName', 'PAbbrev', 'DName', 'DAbbrev', 'DiName', 'DiAbbrev', 'HalfLife', 'HLUnits'])
isotope_info['index'] = ['generic', 'Rb87']
isotope_info['Name'] = ['Generic', 'Rb-87->Sr-87']
isotope_info['PName'] = ['Parent', 'Rubidium-87']
isotope_info['PAbbrev'] = ['P', 'Rb-87']
isotope_info['DName'] = ['Daughter', 'Strontium-87']
isotope_info['DAbbrev'] = ['D', 'Sr-87']
isotope_info['DiName'] = ['Non-Radiogenic Isotope of Daughter Element', 'Strontium-86']
isotope_info['DiAbbrev'] = ['D_i', 'Sr-86']
isotope_info['HalfLife'] = [ 1, 48.8 ]
isotope_info['HLUnits'] = [ 'half-lives', 'Billion years']
isotope_info = isotope_info.set_index('index')

# Set initial isotope to plot
init_isotope = 'generic'

In [4]:
##
## Define the initial amounts of parent and daughter in the sample.
##
## In principle, I would change this depending on the isotopes we plot.  But I am only plotting
## Rb87 --> Sr-87, since that is the most classical use of this Geochron approach.
##

# Range of P to D_i fractions and initial amounts of D to D_i to consider
P2Di_min = 0.05
P2Di_max = 0.40
D2Di0_min = 0.05
D2Di0_max = 0.75

# Generate three mineral samples in different thirds of the entire range
range_P2Di  = (P2Di_max-P2Di_min)

# Create sample amounts
n_samples = 4
nums = np.array(list(range(1, n_samples+1)))
initial_samples = pd.DataFrame(index=nums)
initial_D2Di0 = D2Di0_min + (D2Di0_max - D2Di0_min) * np.random.random()
initial_samples['P2Di'] = P2Di_min + (range_P2Di/n_samples) * (nums - np.random.random(n_samples))
initial_samples['D2Di'] = initial_D2Di0*np.ones_like(nums)


In [5]:
##
## Define functions to call when building interactive plot
##

def amt_left(sample_in, taus):
    # Generate a sample DataFrame after tau half-lifes given an initial DataFrame
    sample = sample_in.copy(deep = True)
    sample['P2Di'] = sample_in['P2Di']*((1/2)**(taus))
    sample['D2Di'] = sample_in['D2Di'] + sample_in['P2Di']*(1 - (1/2)**(taus))
    return sample

def line_points(sample):
    global x_min, x_max, y_min, y_max, initial_D2Di0
    
    # Determine the end points of a line going through the sample points.
    x_range = x_max - x_min
    y_range = y_max - y_min
    
    # Slope (extrapolate from first two points - could be done by a fit to the points)
    slope = (sample['D2Di'][2]-sample['D2Di'][1])/(sample['P2Di'][2]-sample['P2Di'][1])
    y_final = initial_D2Di0 + slope*x_range
    x_points = (x_min, x_max)
    y_points = (initial_D2Di0, y_final)
    return x_points, y_points, slope

def init2current(samples0, samples):
    # Compute the lines connecting initital and final points for plotting
    n_pts = len(samples0)

    xlist = []
    ylist = []
    for pt in range(1, n_pts+1):
        x = np.array([ samples0['P2Di'][pt], samples['P2Di'][pt] ])
        y = np.array([ samples0['D2Di'][pt], samples['D2Di'][pt] ])
        xlist.append(x)
        ylist.append(y)
    
    return(xlist, ylist)
    
def HL_changed(change):
    global isotope, sample, initial_samples, dots_current, line_current, connectors, slope_label
    
    # Determine half-life of this isotope
    idx = (isotope_info.Name == isotope.value)
    HL = float(isotope_info[idx].HalfLife.tolist()[0])
    
    # How many half-lives have passed?  Use this to get new sample and line info
    this_tau = HL_slider.value / HL
    sample = amt_left(initial_samples, this_tau)
    x_sample, y_sample, slope =  line_points(sample)
    
    
    # Update plot
    dots_current.x = sample['P2Di']
    dots_current.y = sample['D2Di']
    line_current.x = x_sample
    line_current.y = y_sample
    slope_label.value = 'Slope: {0:.2f}'.format(slope)
    xlist, ylist = init2current(initial_samples, sample)
    connectors.x = xlist
    connectors.y = ylist
    
    
def isotope_changed(change):
    global ax_x_P2Di, ax_y_D2Di, HL_slider, HLlabel, UnitsText, Max_half_lives

    # Extract the necessary isotope descriptors from the Pandas DataFrame
    idx = (isotope_info.Name == change.new)
    HL = float(isotope_info[idx].HalfLife.tolist()[0])
    HLUnits = isotope_info[idx].HLUnits.tolist()[0]
    PAbbrev = isotope_info[idx].PAbbrev.tolist()[0]
    DAbbrev = isotope_info[idx].DAbbrev.tolist()[0]
    DiAbbrev = isotope_info[idx].DiAbbrev.tolist()[0]

    # Get old half-life
    idx_old = (isotope_info.Name == change.old)
    HL_old = float(isotope_info[idx_old].HalfLife.tolist()[0])

    # Determine current age reading from slider and adjust to new units
    init_age = HL_slider.value 
    
    # Hard code generic versus others
    if (change.new != isotope_info.loc['generic'].Name):
        HL_slider.description = "Time"
    else: 
        HL_slider.description = "Half-lives"    

    # Adjust time scales
    if (HL_old < HL):
        # Adjust maximum limits first before adjusting values (since new HL > old HL)
        HL_slider.max = Max_half_lives*HL
        HLlabel.max = HL_slider.max 
        HL_slider.value = HL*(init_age/HL_old)
        HLlabel.value = HL_slider.value       
    else:
        # Adjust maximum limits after adjusting values (since new HL < old HL)
        HL_slider.value = HL*(init_age/HL_old)
        HLlabel.value = HL_slider.value
        HL_slider.max = Max_half_lives*HL
        HLlabel.max = HL_slider.max 
                
    # Set the axes and other labels to display
    UnitsText.value = HLUnits
    ax_x_P2Di.label = '{0} / {1}'.format(PAbbrev, DiAbbrev)
    ax_y_D2Di.label = '{0} / {1}'.format(DAbbrev, DiAbbrev)



In [11]:
##
## Set up isochron plot
##

# Largest possible fraction of decay (only go out to 5 half-lives)
Max_half_lives = 5
Max_decay_fraction = 1 - (1/2)**(Max_half_lives)

# detemine maximum and minimum values of X and Y axes
x_step = 0.05
x_min = 0
x_max = x_step * ceil(initial_samples['P2Di'][n_samples] / x_step)
y_step = 0.04
y_min = y_step * floor(initial_D2Di0 / y_step)
y_max = y_step * ceil((initial_D2Di0 + initial_samples['P2Di'][n_samples] * Max_decay_fraction) / y_step)

# Labels and scales for Axes
x_P2Di = bq.LinearScale(min = x_min, max = x_max)
y_D2Di = bq.LinearScale(min = y_min, max = y_max)
ax_x_P2Di = bq.Axis(label='P / D_i', scale=x_P2Di)
ax_y_D2Di = bq.Axis(label='D / D_i', scale=y_D2Di, orientation='vertical')

# Set up initial conditions
taus = 0    # zero half lives past
sample = amt_left(initial_samples, taus)

##
## Define the lines
##

# Initial amount of daughter line (with dots for initial amounts of parent)
x_init, y_init, slope_init =  line_points(initial_samples)
line_initial = bq.Lines(x=x_init, y=y_init, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dashed', colors=['red'], labels=['Initial Sample'])
dots_initial = bq.Scatter(x=initial_samples['P2Di'], y=initial_samples['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['white'], stroke='red', fill= True, labels=['Initial Isochron'])

# Current quantities on isochron line
x_sample, y_sample, slope =  line_points(sample)
line_current = bq.Lines(x=x_sample, y=y_sample, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='solid', colors=['red'], labels=['Current Isochron'])
dots_current = bq.Scatter(x=sample['P2Di'], y=sample['D2Di'], scales={'x': x_P2Di, 'y': y_D2Di}, 
                   colors=['red'], stroke='red', fill= True, labels=['Current Isochron'])

# Connect Initial and Current quantities on isochron line
xlist, ylist = init2current(initial_samples, sample)
connectors = bq.Lines(x=xlist, y=ylist, scales={'x': x_P2Di, 'y': y_D2Di}, 
                   line_style='dotted', colors=['black'])

##
## Construct plot
##
isochron = bq.Figure(axes=[ax_x_P2Di, ax_y_D2Di], 
                     marks=[connectors, line_initial, dots_initial, line_current, dots_current],
                     title='Isochron Diagram', 
                     layout={'width': '700px', 'height': '500px', 
                             'max_width': '700px', 'max_height': '500px',
                             'min_width': '600px', 'min_height': '400px'})

##
## Construct controls
##

# Select Generic or Specific Isotopes
isotope = widgets.RadioButtons(options=list(isotope_info.Name), 
                               value=isotope_info.loc[init_isotope].Name, description='Isotope:', 
                               disabled=False, 
                               layout=widgets.Layout(height='75px', max_height='100px', min_height='50px', 
                                                    width='200px', max_width='300px',  min_width='100px'))
isotope.observe(isotope_changed, 'value')

# Slider and text field controling age
idx = (isotope_info.Name == isotope.value)
HL = float(isotope_info[idx].HalfLife.tolist()[0])
HLUnits = isotope_info[idx].HLUnits.tolist()[0]

HL_slider = widgets.FloatSlider(value=0, min=0, max=Max_half_lives*HL, step=0.02,
                                description='Half-lives', disabled=False,
                                continuous_update=True, orientation='horizontal',
                                readout=False, readout_format='.2f',
                                layout=widgets.Layout(height='75px', max_height='100px', min_height='50px', 
                                                    width='200px', max_width='300px',  min_width='100px'))
HL_slider.observe(HL_changed, 'value')

# Get units value and units for age label, then apply them
HLlabel = widgets.BoundedFloatText(value = HL_slider.value, min = HL_slider.min, max = HL_slider.max, 
                                   step = HL_slider.step,
                                       layout={'width': '75px', 'height': '50px', 
                                               'max_width': '75px', 'max_height': '75px',
                                               'min_width': '50px', 'min_height': '50px'})
UnitsText = widgets.Label(value=HLUnits)
age_label = widgets.HBox([HLlabel, UnitsText])
# Link HL slider with this text
widgets.jslink((HL_slider, 'value'), (HLlabel, 'value'))

# Describe slope
slope_label = widgets.Label(value = 'Slope: {0:.2f}'.format(slope),
                                       layout={'align_items':'center','align_content':'center', 
                                               'justify_content':'center', 
                                               'width': '75px', 'height': '50px', 
                                               'max_width': '75px', 'max_height': '75px',
                                               'min_width': '50px', 'min_height': '50px'})


controls = widgets.VBox( [isotope, HL_slider, age_label, slope_label], 
                        layout=widgets.Layout(align_content='center', align_items='center', 
                                              justify_content='center', 
                                              width='300px', height='500px', 
                                              max_width='300px', max_height='500px',
                                              min_width='100px', min_height='400px',
                                              overflow_x='hidden', overflow_y='hidden') )

display(widgets.HBox( [isochron, controls] ) )


